<a href="https://colab.research.google.com/github/parknaeeun/OliveYoung/blob/main/OliveYoung_ranking_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

Data Features
- category: 분류
- rank: 순위
- brand: 브랜드명
- item: 제품명
- fixed_price: 정가
- sale_price: 할인가
- promotion: 프로모션

In [2]:
# 코랩에서 셀레니움 사용 시 설치
# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [18]:
# 드라이버 생성 및 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [19]:
# 올리브영 홈페이지 접근
url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000030006&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EC%B7%A8%EB%AF%B8%2F%ED%8C%AC%EC%8B%9C"
driver.get(url)

print(driver) # 실행 확인

<selenium.webdriver.chrome.webdriver.WebDriver (session="acd3a5cdebd3832074713694a8b1066c")>


In [20]:
# 크롤링
def ranking_crawling(df):
    for c in range(2, 22): #전체 외 20개 카테고리
        category = driver.find_element(By.CSS_SELECTOR, f'#Container > div.best-area > div.common-menu > ul > li.on > button').text

        print(f'{category} 크롤링 시작')
        for i in range(1, 26):
            for j in range(1, 5):
                rank = (i-1)*4 + j
                brand = driver.find_element(By.CSS_SELECTOR, f'#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > div > a > span').text
                item = driver.find_element(By.CSS_SELECTOR, f'#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > div > a > p').text
                # 정가와 행사가가 동일한 제품에 대한 처리
                try:
                    fixed_price = driver.find_element(By.CSS_SELECTOR, f'#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > p.prd_price > span.tx_org > span').text
                except:
                    fixed_price = driver.find_element(By.CSS_SELECTOR, f'#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > p.prd_price > span.tx_cur > span').text
                sale_price = driver.find_element(By.CSS_SELECTOR, f'#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > p.prd_price > span.tx_cur > span').text
                p_span = driver.find_elements(By.CSS_SELECTOR, f'#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > p.prd_flag > span')
                promotion = ([span.text for span in p_span])
                df.loc[len(df)] = [category, rank, brand, item, fixed_price, sale_price, promotion]
                sleep(2)

        print(f'{category} 크롤링 완료')

        page_button = driver.find_element(By.CSS_SELECTOR, f'#Container > div.best-area > div.common-menu > ul > li:nth-child({c+1}) > button')
        page_button.click()
        sleep(2)

In [21]:
# 데이터프레임 형성
df_top_100 = pd.DataFrame(columns=['category', 'rank', 'brand', 'item', 'fixed_price', 'sale_price', 'promotion'])
df_top_100

,category,rank,brand,item,fixed_price,sale_price,promotion


In [ ]:
ranking_crawling(df_top_100)
print(df_top_100)

driver.quit()

In [23]:
df_top_100.to_csv('OliveYoung_top100.csv', index=False)